In [1]:
import pandas as pd
import numpy as np
week1 = pd.read_parquet('/home/cheam/IP_alpha/xgb_rolling_test/results_xgb.parquet')[['mean']]
week2 = pd.read_parquet('/home/cheam/IP_alpha/xgb_rolling_mse/results_xgb.parquet')[['mean']]
ic1 = pd.read_parquet('/home/cheam/IP_alpha/xgb_rolling_test/result_all_train.parquet')[['test_start_date','out_of_sample_ic','in_sample_ic']]
ic1['diff_ic1'] = ic1['out_of_sample_ic'] - ic1['in_sample_ic']
ic1.rename(columns={'out_of_sample_ic':'ic1_out_of_sample','in_sample_ic':'ic1_in_sample'},inplace=True)
ic2 = pd.read_parquet('/home/cheam/IP_alpha/xgb_rolling_mse/result_all_train.parquet')[['out_of_sample_ic','in_sample_ic']]
ic2['diff_ic2'] = ic2['out_of_sample_ic'] - ic2['in_sample_ic']
ic2.rename(columns={'out_of_sample_ic':'ic2_out_of_sample','in_sample_ic':'ic2_in_sample'},inplace=True)
ic = pd.concat([ic1, ic2], axis=1)
ic.set_index('test_start_date',inplace=True)

# 挑选两个模型中diff更小的组成新的模型

In [14]:
# 挑选diff_ic 小的模型
model1 = pd.DataFrame()
performance1 = pd.DataFrame(index = ic.index, columns=['out_of_sample_ic','in_sample_ic','diff_ic'])
for i in range(len(ic)):
    if i == len(ic) - 1:
        current_index = ic.index[i]
        if abs(ic['diff_ic1'].iloc[i]) < abs(ic['diff_ic2'].iloc[i]):
        # 输出 week1 中该 ic 中 index 和下一个 index 之间的数据
            data_between = week1.loc[current_index:]
            performance1.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic1_out_of_sample']
            performance1.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic1_in_sample']
            performance1.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic1']
            model1 = pd.concat([model1, data_between])
        else:
            data_between = week2.loc[current_index:]
            performance1.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic2_out_of_sample']
            performance1.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic2_in_sample']
            performance1.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic2']
            model1 = pd.concat([model1, data_between])
    else:
        current_index = ic.index[i]
        next_index = ic.index[i + 1]

        # 比较 diff1 和 diff2
        if abs(ic['diff_ic1'].iloc[i]) < abs(ic['diff_ic2'].iloc[i]):
            # 输出 week1 中该 ic 中 index 和下一个 index 之间的数据
            data_between = week1.loc[current_index:next_index]
            performance1.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic1_out_of_sample']
            performance1.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic1_in_sample']
            performance1.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic1']
            model1 = pd.concat([model1, data_between])
        else:
            data_between = week2.loc[current_index:next_index]
            performance1.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic2_out_of_sample']
            performance1.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic2_in_sample']
            performance1.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic2']
            model1 = pd.concat([model1, data_between])
    

In [15]:
df_unique = model1[~model1.index.duplicated(keep='first')]
df_unique

mean
Date       Symbol             
2012-01-02 000001.SZ -0.000074
           000002.SZ -0.000076
           000004.SZ -0.000074
           000005.SZ -0.000074
           000006.SZ -0.000076
...                        ...
2024-09-23 688799.SH  0.002961
           688800.SH  0.003138
           688819.SH  0.001406
           688981.SH  0.001713
           689009.SH  0.001939

[2360004 rows x 1 columns]

# 选择out_ic大的

In [9]:
# 挑选diff_ic 小的模型
model2 = pd.DataFrame()
performance2 = pd.DataFrame(index = ic.index, columns=['out_of_sample_ic','in_sample_ic','diff_ic'])
for i in range(len(ic)):
    if i == len(ic) - 1:
        current_index = ic.index[i]
        if abs(ic['ic1_out_of_sample'].iloc[i]) > abs(ic['ic2_out_of_sample'].iloc[i]):
        # 输出 week1 中该 ic 中 index 和下一个 index 之间的数据
            data_between = week1.loc[current_index:]
            performance2.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic1_out_of_sample']
            performance2.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic1_in_sample']
            performance2.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic1']
            model2 = pd.concat([model2, data_between])
        else:
            data_between = week2.loc[current_index:]
            performance2.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic2_out_of_sample']
            performance2.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic2_in_sample']
            performance2.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic2']
            model2 = pd.concat([model2, data_between])
    else:
        current_index = ic.index[i]
        next_index = ic.index[i + 1]

        # 比较 diff1 和 diff2
        if abs(ic['ic1_out_of_sample'].iloc[i]) > abs(ic['ic2_out_of_sample'].iloc[i]):
            # 输出 week1 中该 ic 中 index 和下一个 index 之间的数据
            data_between = week1.loc[current_index:next_index]
            performance2.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic1_out_of_sample']
            performance2.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic1_in_sample']
            performance2.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic1']
            model2 = pd.concat([model2, data_between])
        else:
            data_between = week2.loc[current_index:next_index]
            performance2.loc[current_index]['out_of_sample_ic'] = ic.loc[current_index]['ic2_out_of_sample']
            performance2.loc[current_index]['in_sample_ic'] = ic.loc[current_index]['ic2_in_sample']
            performance2.loc[current_index]['diff_ic'] = ic.loc[current_index]['diff_ic2']
            model2 = pd.concat([model2, data_between])
    

In [10]:
df_unique = model2[~model2.index.duplicated(keep='first')]
df_unique

mean
Date       Symbol             
2012-01-02 000001.SZ  0.000379
           000002.SZ -0.000669
           000004.SZ  0.000965
           000005.SZ  0.000758
           000006.SZ  0.000439
...                        ...
2024-09-23 688799.SH  0.004419
           688800.SH  0.004452
           688819.SH  0.001803
           688981.SH  0.001213
           689009.SH  0.002341

[2360004 rows x 1 columns]